## 1. Exploratory Data Analysis (EDA)

### Importar módulos y librerías

In [23]:
import ast 
# módulo ast: helps Python applications to process trees of the Python abstract syntax grammar.
# The abstract syntax itself might change with each Python release; 
# this module helps to find out programmatically what the current grammar looks like.

import pandas as pd
# librería pandas: diseñada para la manipulación y el análisis de datos.
# El nombre «Pandas» es en realidad una contracción del término «Panel Data» para series de datos 
# que incluyen observaciones a lo largo de varios periodos de tiempo.
# pandas trabaja sobre dataframes: tablas de datos bidimensionales, donde cada columna contiene 
# los valores de una variable y cada fila contiene un conjunto de valores de cada columna.

import seaborn as sns
# librería seaborn: para visualización de datos, basada en matplotlib

import matplotlib.pyplot as plt
# módulo pyplot de la librería matplotlib

from datasets import load_dataset
# librería datasets: para cargar y manipular datasets de Hugging Face

### Cargar datos

In [40]:
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()
# Conversión del dataset a un DataFrame de pandas

In [25]:
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Senior Data Engineer,Senior Clinical Data Engineer / Principal Clin...,"Watertown, CT",via Work Nearby,Full-time,False,"Texas, United States",2023-06-16 13:44:15,False,False,United States,None,NaN,NaN,Boehringer Ingelheim,None,None
1,Data Analyst,Data Analyst,"Guadalajara, Jalisco, Mexico",via BeBee México,Full-time,False,Mexico,2023-01-14 13:18:07,False,False,Mexico,None,NaN,NaN,Hewlett Packard Enterprise,"['r', 'python', 'sql', 'nosql', 'power bi', 't...","{'analyst_tools': ['power bi', 'tableau'], 'pr..."
2,Data Engineer,"Data Engineer/Scientist/Analyst, Mid or Senior...","Berlin, Germany",via LinkedIn,Full-time,False,Germany,2023-10-10 13:14:55,False,False,Germany,None,NaN,NaN,ALPHA Augmented Services,"['python', 'sql', 'c#', 'azure', 'airflow', 'd...","{'analyst_tools': ['dax'], 'cloud': ['azure'],..."
3,Data Engineer,LEAD ENGINEER - PRINCIPAL ANALYST - PRINCIPAL ...,"San Antonio, TX",via Diversity.com,Full-time,False,"Texas, United States",2023-07-04 13:01:41,True,False,United States,None,NaN,NaN,Southwest Research Institute,"['python', 'c++', 'java', 'matlab', 'aws', 'te...","{'cloud': ['aws'], 'libraries': ['tensorflow',..."
4,Data Engineer,Data Engineer- Sr Jobs,"Washington, DC",via Clearance Jobs,Full-time,False,Sudan,2023-08-07 14:29:36,False,False,Sudan,None,NaN,NaN,Kristina Daniel,"['bash', 'python', 'oracle', 'aws', 'ansible',...","{'cloud': ['oracle', 'aws'], 'other': ['ansibl..."


### Limpieza de datos

In [26]:
type(df.job_posted_date[0])
# La columna job_posted_date es un objeto de tipo string

str

In [41]:
# Convertir la columna job_posted_date a tipo datetime y guardar esos cambios en el DataFrame
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

# Convertir la columna job_skills de string a lista
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

# Si no se utiliza apply, se generaría un error al intentar aplicar ast.literal_eval a toda la columna.

In [38]:
# Para chequear que la conversión se ha realizado correctamente
df['job_skills'][1]

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [39]:
type(df['job_skills'][1])

list